ICAを用いて、神経画像データのどの信号が本物の信号かを推定する。

In [1]:
%matplotlib inline

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltools.data import Brain_Data
from nltools.plotting import component_viewer

base_dir = '/Users/hidekiizumi/myproject/Localizer/derivatives/fmriprep'
sub = 'S01'

data = Brain_Data(os.path.join(base_dir, f'sub-{sub}','func', f'sub-{sub}_task-localizer_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'))

低周波のデータを除去するため、ハイパスヒィルターをかける。さらに、ガウシアンカーネルを用いて、データを平滑化する。

In [4]:
data = data.filter(sampling_freq=1/2.4, high_pass=1/128)

data = data.smooth(6)

ICA : 多変量信号を、サブコンポーネントに分解する手法。信号の独立性を仮定する。

In [3]:
tr = 2.4

# 30個のコンポーネントを抽出
output = data.decompose(algorithm='ica', n_components=30, axis='images', whiten=True)

/Users/hidekiizumi/miniforge3/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:679: FutureWarning: Starting in v1.3, whiten=True should be specified as whiten='arbitrary-variance' (its current behaviour). This behavior is deprecated in 1.1 and will raise ValueError in 1.3.
  self._fit(X, compute_sources=False)


分解した信号の可視化を行う。

- プロット１：コンポーネント毎に、その位置での信号強度が得られる。時間的変化の具合は標準偏差で見られるため、可視化における標準偏差の閾値を設定することで、どこが刺激に対してよく活動しているか見やすくなる。？
- プロット２：横軸は時間、縦軸はそのコンポーネントにおけるボクセルの信号強度。コンポーネント毎の信号強度の時間的変化を見ることができる。？
- プロット３：一定時間間隔TR = 2.4sで観測された脳の信号強度をフーリエ変換で時間領域から周波数領域に変換し、その周波数領域でのパワーとしてプロットする。コンポーネント毎に、どの周波数帯域で活動しているかを見ることができる。周波数の可視化可能な領域は、ナイキスト周波数（サンプリング周波数の半分）で、今回は1/(2.4*2) = 0.21Hzまでであり、加えてハイパスフィルタで0.0078Hz以下の信号を除去しているため、0.0078Hzから0.21Hzまでの周波数帯域での信号強度を見ることができる。これより速い周波数で振動する信号（呼吸や心臓の拍）は、エイリアシングが発生する。

In [6]:
component_viewer(output, tr=2.4)

interactive(children=(BoundedIntText(value=0, description='Component', max=29), BoundedFloatText(value=2.0, de…

## 練習問題

【問題】

被験者s01とs02に対して、どのコンポーネントが信号で、どのコンポーネントがノイズであるかを推測してみる。また、見つけたノイズのタイプもラベル付けしてみる（例：頭部の動き、スキャナーのスパイク、心臓、呼吸など）。

この判断をする際に重要だと思われる特徴は何か？空間マップは何か有用な情報を提供するか？コンポーネントの時間経過についてはどうか？それはタスクのありそうな時間経過に対応しているか。パワースペクトラムについてはどうか？


【回答】

まずタスクがわからないので、解剖学的マップ見ても何もわからない。時間経過は周期的だと心拍や呼吸に関連している可能性、突然上がったり下がってたりしてるだけだと頭部の動きの可能性、などがあるが、目視で確認は正直難しい。パワースペクトラムは、以下のポイントを参考にすると幾らか役に立つ可能性あり。

- 頭部の動き：一般的に周波数は非常に低い。これは、被験者がスキャナー内で一定の位置を保つことが求められ、可能な限り動かないように指導されるため。ただし、被験者が不随意に微妙に動く（例えば、呼吸や心拍に連動した微小な動き）場合があり、その周波数は呼吸や心拍と関連している可能性がある。ここではハイパスフィルタで除去したいくらい低いものと考える。

- スキャナースパイク（電子的なノイズ）：多分低い

- 呼吸：通常の安静時の呼吸率は大体毎分12〜20回。これをHz（1秒あたりの回数）で表すと、大体0.2〜0.33Hz。

- 心臓の脈拍（心拍数）は、大人の安静時で通常は毎分60〜100回。これを秒単位に直すと、1秒間に約1〜1.67回の脈拍。これは周波数として約1〜1.67Hz。

### S01

回答テンプレート（面倒すぎたので飛ばした）

- 信号：
- ノイズ
  - 頭部の動き：
  - スキャナースパイク（電子的なノイズ）：
  - 呼吸：
  - 心臓の脈拍：

In [10]:
sub = 'S01'
data = Brain_Data(os.path.join(base_dir, f'sub-{sub}','func', f'sub-{sub}_task-localizer_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'))
data = data.filter(sampling_freq=1/2.4, high_pass=1/128)
data = data.smooth(6)
tr = 2.4
output = data.decompose(algorithm='ica', n_components=30, axis='images', whiten=True)

component_viewer(output, tr=2.4)

/Users/hidekiizumi/miniforge3/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:679: FutureWarning: Starting in v1.3, whiten=True should be specified as whiten='arbitrary-variance' (its current behaviour). This behavior is deprecated in 1.1 and will raise ValueError in 1.3.
  self._fit(X, compute_sources=False)


interactive(children=(BoundedIntText(value=0, description='Component', max=29), BoundedFloatText(value=2.0, de…

### S02

回答テンプレート（面倒すぎたので飛ばした）

- 信号：
- ノイズ
  - 頭部の動き：
  - スキャナースパイク（電子的なノイズ）：
  - 呼吸：
  - 心臓の脈拍：

In [7]:
sub = 'S02'
data = Brain_Data(os.path.join(base_dir, f'sub-{sub}','func', f'sub-{sub}_task-localizer_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'))
data = data.filter(sampling_freq=1/2.4, high_pass=1/128)
data = data.smooth(6)
tr = 2.4
output = data.decompose(algorithm='ica', n_components=30, axis='images', whiten=True)

component_viewer(output, tr=2.4)

/Users/hidekiizumi/miniforge3/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:679: FutureWarning: Starting in v1.3, whiten=True should be specified as whiten='arbitrary-variance' (its current behaviour). This behavior is deprecated in 1.1 and will raise ValueError in 1.3.
  self._fit(X, compute_sources=False)


interactive(children=(BoundedIntText(value=0, description='Component', max=29), BoundedFloatText(value=2.0, de…